In [1]:
%env MUJOCO_GL=egl
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.interpolate import CubicSpline, PchipInterpolator
import os
import copy
import pickle 
from loco_mujoco import LocoEnv
from mushroom_rl.utils.record import VideoRecorder
from tqdm.notebook import tqdm
import mujoco

env: MUJOCO_GL=egl


In [3]:
# with open('locomujoco_transformed_data.pkl', 'rb') as f:
with open('locomujoco_transformed_data_55_speeds_0025_increment.pkl', 'rb') as f:
    data = pickle.load(f)

In [4]:
dt = 0.01

In [5]:
simulation_data = {}
for speed in tqdm(data):
    simulation_data[speed] = {}
    for joint in data[speed]:
        concat_data = np.concatenate(data[speed][joint])
        if joint not in ['q_pelvis_tx', 'q_pelvis_tz', 'q_pelvis_ty']:
            concat_data = np.deg2rad(concat_data)
        if 'knee' in joint:
            concat_data = -1*concat_data
        
        simulation_data[speed][joint] = concat_data   
        simulation_data[speed]['d'+joint] = np.gradient(concat_data, dt)   

100%|██████████████████████████████| 55/55 [00:00<00:00, 63.26it/s]


In [7]:
ignore_keys = ["q_pelvis_tx", "q_pelvis_tz"]

In [8]:
joints = [joint[0] for joint in mdp.obs_helper.observation_spec]
print(joints)

['q_pelvis_tx', 'q_pelvis_tz', 'q_pelvis_ty', 'q_pelvis_tilt', 'q_pelvis_list', 'q_pelvis_rotation', 'q_hip_flexion_r', 'q_hip_adduction_r', 'q_hip_rotation_r', 'q_knee_angle_r', 'q_ankle_angle_r', 'q_hip_flexion_l', 'q_hip_adduction_l', 'q_hip_rotation_l', 'q_knee_angle_l', 'q_ankle_angle_l', 'q_lumbar_extension', 'q_lumbar_bending', 'q_lumbar_rotation', 'dq_pelvis_tx', 'dq_pelvis_tz', 'dq_pelvis_ty', 'dq_pelvis_tilt', 'dq_pelvis_list', 'dq_pelvis_rotation', 'dq_hip_flexion_r', 'dq_hip_adduction_r', 'dq_hip_rotation_r', 'dq_knee_angle_r', 'dq_ankle_angle_r', 'dq_hip_flexion_l', 'dq_hip_adduction_l', 'dq_hip_rotation_l', 'dq_knee_angle_l', 'dq_ankle_angle_l', 'dq_lumbar_extension', 'dq_lumbar_bending', 'dq_lumbar_rotation']


In [10]:
training_data = {}
for speed in simulation_data:
    training_data[speed] = {}
    all_states = np.stack([simulation_data[speed][joint] for joint in joints if joint not in ignore_keys]).T
    training_data[speed]['states'] = all_states[:-1, :]
    training_data[speed]['next_states'] = all_states[1:, :]
    training_data[speed]['absorbing'] = np.zeros(89799)
    training_data[speed]['last'] = np.concatenate([np.zeros(89799), np.ones(1)])

In [11]:
with open('locomujoco_speeds_dataset_55_speeds_0025_increment.pkl', 'wb') as f:
    pickle.dump(training_data, f)

In [12]:
training_data.keys()

dict_keys([0.5, 0.525, 0.55, 0.575, 0.6, 0.625, 0.65, 0.675, 0.7, 0.725, 0.75, 0.775, 0.8, 0.825, 0.85, 0.875, 0.9, 0.925, 0.95, 0.975, 1.0, 1.025, 1.05, 1.075, 1.1, 1.125, 1.15, 1.175, 1.2, 1.225, 1.25, 1.275, 1.3, 1.325, 1.35, 1.375, 1.4, 1.425, 1.45, 1.475, 1.5, 1.525, 1.55, 1.575, 1.6, 1.625, 1.65, 1.675, 1.7, 1.725, 1.75, 1.775, 1.8, 1.825, 1.85])